In [1]:

!pip install --upgrade transformers sentencepiece datasets sacrebleu rouge-score accelerate --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 23.4 MB/s eta 0:00:00


In [4]:
import sys
import pandas as pd
import torch
from datasets import Dataset, DatasetDict

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

print("Python:", sys.version.splitlines()[0])
print("Transformers version:", transformers.__version__)


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Transformers version: 4.57.1


In [7]:
!pip install chardet


In [22]:
!pip install chardet
import chardet

def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read())
    return result

print(detect_encoding("/content/vaders_sentiment_25k.csv"))



{'encoding': 'MacRoman', 'confidence': 0.7125481914243206, 'language': ''}


In [23]:
import csv

# Load tel1.csv safely
rows1 = []
with open("/content/vaders_sentiment_25k.csv", "r", encoding="utf-8", errors="ignore") as f:
    reader = csv.reader(f)
    for row in reader:
        rows1.append(row)

tel1_df = pd.DataFrame(rows1[1:], columns=rows1[0])
print("tel1 loaded:", tel1_df.shape)
print("Columns:", tel1_df.columns)





tel1 loaded: (25000, 7)
Columns: Index(['Unnamed: 0', 'neg', 'neu', 'pos', 'compound', 'statement', 'status'], dtype='object')


In [10]:
# Rename columns
tel1_df = tel1_df.rename(columns={"input": "prompt", "output": "response"})


# Drop the 'instruction' column
tel1_df = tel1_df.drop(columns=["instruction"])


# Check results
print(tel1_df.head())



KeyError: "['instruction'] not found in axis"

In [24]:
print(tel1_df.columns)


Index(['Unnamed: 0', 'neg', 'neu', 'pos', 'compound', 'statement', 'status'], dtype='object')


In [25]:
tel1_df = tel1_df.rename(columns={
    "statement": "prompt",
    "status": "response"
})


In [26]:
tel1_df = tel1_df.drop(columns=["Unnamed: 0"])



In [27]:
# Check the result
print(tel1_df.head())
print(tel1_df.columns)


     neg    neu    pos compound  \
0  0.228  0.619  0.153   -0.933   
1  0.346  0.654    0.0  -0.5719   
2  0.244   0.73  0.025  -0.9287   
3  0.078  0.724  0.198   0.8313   
4    0.0  0.566  0.434   0.5574   

                                              prompt    response  
0  Just as the the title says. I feel like one is...  Depression  
1  a blackened sky encroached tugging behind it m...  Depression  
2  It gives you insomnia, which in turn makes you...  Depression  
3  Hello all, I'm a new submitter to this channel...      Normal  
4                   Thank God the CB is over for Eid      Normal  
Index(['neg', 'neu', 'pos', 'compound', 'prompt', 'response'], dtype='object')


In [28]:
df = pd.concat([tel1_df], ignore_index=True)

# Remove duplicate prompts
df = df.drop_duplicates(subset=["prompt"])

# Remove empty rows
df = df.dropna(subset=["prompt", "response"])

# Shuffle
df = df.sample(frac=1).reset_index(drop=True)

print("Final merged dataset size:", df.shape)
df.head()


Final merged dataset size: (24394, 6)


,neg,neu,pos,compound,prompt,response
0,0.0,1.0,0.0,0.0,"Place: 12:32:57, 13.75C.",Normal
1,0.0,1.0,0.0,0.0,the sun is shining got ta get to work,Normal
2,0.425,0.575,0.0,-0.5719,cassou post concert depression tu connais,Depression
3,0.0,0.734,0.266,0.504,"[bb] If after waxing, what should I wear? or w...",Normal
4,0.233,0.583,0.184,-0.6121,I wish that we all were equipped with a Self D...,Suicidal


In [29]:
train_df = df.sample(frac=0.8, random_state=42)
remaining = df.drop(train_df.index)

val_df = remaining.sample(frac=0.5, random_state=42)
test_df = remaining.drop(val_df.index)

print("Train:", train_df.shape)
print("Val:", val_df.shape)
print("Test:", test_df.shape)


Train: (19515, 6)
Val: (2440, 6)
Test: (2439, 6)


In [30]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True))
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['neg', 'neu', 'pos', 'compound', 'prompt', 'response'],
        num_rows: 19515
    })
    validation: Dataset({
        features: ['neg', 'neu', 'pos', 'compound', 'prompt', 'response'],
        num_rows: 2440
    })
    test: Dataset({
        features: ['neg', 'neu', 'pos', 'compound', 'prompt', 'response'],
        num_rows: 2439
    })
})


In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/mt5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Tokenizer and Model Loaded!")


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Tokenizer and Model Loaded!


In [32]:
def preprocess(batch):
    inputs = tokenizer(
        batch["prompt"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = tokenizer(
        batch["response"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    inputs["labels"] = labels["input_ids"]
    return inputs


In [33]:
tokenized_ds = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print("Tokenization Completed!")


Map:   0%|          | 0/19515 [00:00<?, ? examples/s]

Map:   0%|          | 0/2440 [00:00<?, ? examples/s]

Map:   0%|          | 0/2439 [00:00<?, ? examples/s]

Tokenization Completed!


In [34]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)


In [38]:
from transformers import TrainingArguments

batch_size = 4
num_epochs = 3
output_dir = "./eng_chatbot_model"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,

    # 🚀 FIX: disable wandb
    report_to="none"
)

print("TrainingArguments Ready!")


TrainingArguments Ready!


In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

print("Trainer Ready!")


/tmp/ipython-input-483616945.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer Ready!


In [40]:
import os, sys, types
os.environ["WANDB_DIABLED"] = "true"
os.environ["WANDB_SILENT"] = "true"
os.environ["WANDB_MODE"] = "disabled"

fake_wandb = types.SimpleNamespace(
    init= lambda *a, **k:None,
    log=lambda*a, **k:None,
    finish=lambda *a, **k:None,
    run=None,
    config={}
)
sys.modules["wandb"]=fake_wandb
print("disabled")

disabled


In [41]:
!pip uninstall -y wandb

Found existing installation: wandb 0.22.3
Uninstalling wandb-0.22.3:
  Successfully uninstalled wandb-0.22.3


In [42]:
trainer.train()


Step,Training Loss
50,26.555700
100,6.710300
150,0.981600
200,0.434400
250,0.271200
300,0.251200
350,0.221000
400,0.180500
450,0.121300
500,0.127100


KeyboardInterrupt: 

In [43]:
metrics = trainer.evaluate(tokenized_ds["test"])
metrics


{'eval_loss': 0.004735185764729977}

In [44]:
trainer.save_model("./final_english_chatbot_tel1")
tokenizer.save_pretrained("./final_english_chatbot_tel1")

print("Model saved successfully!")


Model saved successfully!


In [45]:
def en_chatbot(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(model.device)

    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        min_length=5,
        num_beams=4,
        no_repeat_ngram_size=3,
        repetition_penalty=2.0,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [50]:
print("ENG Chatbot Ready! Type 'exit' to quit.\n")

while True:
    msg = input("YOU: ")
    if msg.lower() in ["exit", "quit", "stop"]:
        break
    print("CHATBOT:", en_chatbot(msg))


ENG Chatbot Ready! Type 'exit' to quit.

YOU: a blackened sky encroached tugging behind it my depression
CHATBOT: Personality Depression
YOU: im stressed
CHATBOT: Anxiety <extra_id_11><0x57> disorder
YOU: quit
